<a href="https://colab.research.google.com/github/yoavdana/computational-physics-4/blob/main/comp_phy4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

computational physics EX4 -Yoav Dana

In [10]:
import numpy as np
from matplotlib import pyplot as plt



part 1: jacobian numerical calculation and NR root solver

In [20]:
def numerical_jacobian(f, xs, dx=1e-8):

    ys = f(xs)

    x_dim = xs.shape[0]
    y_dim = ys.shape[0]

    jac = np.empty((y_dim, x_dim))

    for i in range(x_dim):
        part_der_element = e(x_dim, i + 1)
        x_plus = xs + dx * part_der_element
        x_minus = xs - dx * part_der_element
        jac[:, i] = (f(x_plus) - f(x_minus)) / (2 * dx)

    return jac


def e(n, i):
  ret = np.zeros(n)
  ret[i - 1] = 1.0
  return ret


def linear_search(F,x_old,delta):
    g=lambda lamb: np.linalg.norm(F(x_old+lamb*delta),ord=2)
    J = numerical_jacobian(F, x_old, dx=1e-8)
    gra_g0=F(x_old).dot(J).dot(delta)
    g0,g1=g(0),g(1)
    l=-gra_g0/(2*(g1-g0-gra_g0))

    return max(l,0.1)


def Newton_Rap(F, x, eps,lr):
    e=1e-8
    F_value = F(x)
    F_norm = np.linalg.norm(F_value, ord=2)  # l2 norm of vector
    iteration_counter = 0
    while abs(F_norm) > eps and iteration_counter < 5000:
        J = numerical_jacobian(F, x, dx=1e-8)
        delta = -lr*np.matmul(np.linalg.inv(J+e*np.eye(J.shape[0],J.shape[1])),F(x))
        x_temp = x + delta
        F_old,F_temp =np.linalg.norm(F(x), ord=2),np.linalg.norm(F(x_temp), ord=2)
        if np.abs(F_old)>np.abs(F_temp):
            x=x_temp
            F_norm=F_temp
        else:
            lamb=linear_search(F, x, delta)
            x=x+lamb*delta
            F_norm=np.linalg.norm(F(x), ord=2)


        iteration_counter += 1


    return x, iteration_counter


Jacobian calculation test: i choose a few functions with a known jacobian and tested the function "numerical_jacobian"

In [19]:

def f1(x):

    f1 =x[0]
    f2 =5*x[2]
    f3=4*x[1]**2-2*x[2]
    f4=x[2]*np.sin(x[0])

    return np.array([f1,f2,f3,f4])

def real_jac1(x):
    return np.array([[1,0,0],[0,0,5],[0,8*x[1],-2],[x[2]*np.cos(x[0]),0,np.sin(x[0])]])


def f2(x):

    f1 = x[0]**2-x[1]*x[2]
    f2 = x[0]*x[1]*x[2]

    return np.array([f1, f2])

def real_jac2(x):
    return np.array([[2*x[0],-x[2],-x[1]],[x[1]*x[2],x[0]*x[2],x[0]*x[1]]])


x = np.array([1,4,2], dtype=float)

print('numerical jacobian=',numerical_jacobian(f1, x, dx=1e-8))
print('real jacobian=',real_jac1(x))

print('numerical jacobian=',numerical_jacobian(f2, x, dx=1e-8))
print('real jacobian=',real_jac2(x))


def test_Newton_system1():
    from numpy import cos, sin, pi, exp

    def F(x):
        return np.array(
            [x[0] ** 2 - x[1] + x[0] * cos(pi * x[0]),x[0] * x[1] + exp(-x[1]) - x[0] ** (-1)])

    def F1(x):
        x1,x2,x3=x[0],x[1],x[2]
        f1=x1+x2+x3**2-12
        f2=x1-x2+x3-2
        f3=2*x1-x2**2+x3-1
        return np.array([f1,f2,f3])
    expected = np.array([1, 0])


    x, n = Newton_Rap(F, np.array([30, 50],dtype='float64'), 1e-6,0.1)
    print(n, F(x))



    x, n = Newton_Rap(F1, np.array([500,-100, 1000],dtype='float64'), 1e-6,0.1)
    print(n, F1(x))
test_Newton_system1()

numerical jacobian= [[ 1.          0.          0.        ]
 [ 0.          0.          5.00000006]
 [ 0.         31.99999981 -2.00000017]
 [ 1.08060462  0.          0.84147098]]
real jacobian= [[ 1.          0.          0.        ]
 [ 0.          0.          5.        ]
 [ 0.         32.         -2.        ]
 [ 1.08060461  0.          0.84147098]]
numerical jacobian= [[ 1.99999999 -1.99999999 -3.99999998]
 [ 8.          1.99999999  3.99999998]]
real jacobian= [[ 2. -2. -4.]
 [ 8.  2.  4.]]
202 [-8.03354553e-07  5.29106117e-07]
268 [ 9.01885036e-07  1.05525810e-09 -8.03336277e-08]
